このページでは簡易的なコードでゲノミック予測を実施できます。

# Install library & get modules

解析の所で動かしている自作関数の詳細が知りたい場合は、下記コマンドでダウンロードされる`train_model.py`により詳細なコードが書いてあります。

ダウンロードしたファイルは左端のファイルメニューから確認可能です。

(https://github.com/slt666666/shared_GP に全てのコード・スクリプト・サンプルデータは置いてあります。)


In [ ]:
!pip install optuna
!pip install pyper
!wget https://raw.githubusercontent.com/slt666666/shared_GP/refs/heads/main/train_model.py
!Rscript -e 'install.packages("rrBLUP", repos="https://cloud.r-project.org")'

# Get sample dataset

左端のファイルメニューから、自分のPhenotypeデータやGenotypeデータをアップロードして実施することもできます。(その場合は適宜以降のコードのファイル名を変えるか、アップロードするファイル名を`genotype.csv`と`phenotype.csv`にしてください。)

In [ ]:
!wget https://raw.githubusercontent.com/slt666666/shared_GP/refs/heads/main/phenotype.csv
!wget https://raw.githubusercontent.com/slt666666/shared_GP/refs/heads/main/genotype.csv

# Load library

In [ ]:
import copy
import ast
import gzip
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from glob import glob
from tqdm import tqdm

from train_model import make_data, tuning_ElasticNet, train_Enet, train_GBLUP, GBLUP_coef

# Parameter tuning

今回はGBLUPモデルとElasticNetモデルを使用。

ElasticNetはパラメータの指定が必要なのでここでチューニングしています。

In [ ]:
genotype = "genotype.csv"
phenotype = "phenotype.csv"
CV=5
optuna_seed=618
trait_names=["biomass", "grain_number"]

# parameter tuning for each trait
for trait_name in trait_names:

    # make test data & training data by 5fold-CV
    test_data, train_data, position_data = make_data(trait_name, genotype, phenotype, CV, seed=1024)

    # parameter tuning for each CV dataset
    tuning_params = []
    for i in range(CV):

        # parameter tuning
        CV_params = tuning_ElasticNet(train_data[i][0], train_data[i][1])
        tuning_params.append([trait_name, i, CV_params])

    # save tuning parameters
    pd.DataFrame(tuning_params).to_csv("Enet_{}_params.csv".format(trait_name))

# Train models & check accuracy

5-fold cross validationでモデル学習 & 予測精度の検証

In [ ]:
def train_multi(inputs):
    model = inputs[4]
    print(f"Model {model} start")
    if model == "GBLUP":
        test_pred, estimated_h2, r2 = train_GBLUP(inputs[1], inputs[2], inputs[3])
        coefs = GBLUP_coef(inputs[2], inputs[3])
    elif model == "Enet":
        test_pred, r2, coefs = train_Enet(inputs[1], inputs[2], inputs[3])
    return [inputs[0], inputs[1][1].values, test_pred, r2, coefs]

In [ ]:
genotype = "genotype.csv"
phenotype = "phenotype.csv"
CV=5
optuna_seed=1024
trait_names = ["biomass", "grain_number"]
models = ["GBLUP", "Enet"]


# train models for each trait & each statistical model
for trait_name in trait_names:

    print(trait_name)

    summary = []

    # make test & training dataset
    test_data, train_data, position_data = make_data(trait_name, genotype, phenotype, CV, seed=1024, plot=False)

    # train model for each CV data & each model
    for model in models:

        # read parameter
        if model != "GBLUP":
            params = pd.read_csv("{}_{}_params.csv".format(model, trait_name), index_col=0)
            params.columns = ["trait", "CV", "params"]
            params = ast.literal_eval(params.loc[(params["trait"] == trait_name) & (params["CV"] == i), "params"].values[0])
        else:
            params = genotype

        # for each CV
        for i in range(CV):

            # train model
            result = train_multi([i, test_data[i], train_data[i], params, model])

            # add result to summary
            tmp_result = [trait_name, model]
            tmp_result.extend(result)
            summary.append(tmp_result)

    with gzip.open(f'{trait_name}.pic.bin.gz', 'wb') as p:
        pickle.dump(summary, p)

# check results

テストデータにおける予測精度を下記結果で`形質名_train_result.csv`というファイルに保存します。

* 1列目...形質名
* 2列目...使用した統計モデル
* 3列目...テストデータの実測値
* 4列目...テストデータの予測値
* 5列目...実測値と予測値の相関係数
* 6列目...各SNP(変数)の回帰係数

In [ ]:
trait_names = ["biomass", "grain_number"]

summary = pd.DataFrame()
for trait_name in trait_names:
    print(trait_name)
    with gzip.open(f'{trait_name}.pic.bin.gz', 'rb') as p:
        tmp = pd.DataFrame(pickle.load(p))
        tmp.columns = ["trait", "model", "CV", "test_observed", "test_preds", "r2", "coefs"]
    tmp.to_csv(f'{trait_name}_train_result.csv', index=None)
    display(tmp)